<a href="https://colab.research.google.com/github/shardulburde/Exotic-Options-Pricing/blob/main/AsianOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [276]:
#Reading Excel Data
#Excel sheet link in next comment
#https://drive.google.com/file/d/1UXq6kIWAaGUmzk41B4ReCg5L_k_vBtas/view?usp=sharing

import pandas as pd
xls = pd.ExcelFile('/content/drive/MyDrive/IndusInd/Exotic Options.xlsx')
df = pd.read_excel(xls, 'AsianData')
df1 = pd.read_excel(xls, 'MktData')

In [277]:
df

,Unnamed: 0,Arithmetic Average,Unnamed: 2,Geometric Average,Unnamed: 4
0,NaN,Avg Asset,Avg Strike,Avg Asset,Avg Strike
1,Strike,74,NaN,74,NaN
2,Spot,73.32,73.32,73.32,73.32
3,Start Date,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00
4,End Date,2022-06-21 00:00:00,2022-06-22 00:00:00,2022-06-23 00:00:00,2022-06-24 00:00:00
5,Tenor(days),365,365,365,365
6,Tenor(years),1,1,1,1
7,INR Rate(%),0.0425,0.0425,0.0425,0.0425
8,USD Rate(%),0.002,0.002,0.002,0.002
9,Forward,76.3504,76.3504,76.3504,76.3504


In [278]:
#Interpolating Market Vol Data
from scipy import interpolate

def atm(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,4,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def rr25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,7,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def fly25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,10,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)


def rr10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,13,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)

  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)
      

def fly10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,16,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
    
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

In [279]:
#Making Call Delta functions that return vol for given tenor from mkt data

def d10(tenor):
  vol=atm(tenor)+fly10d(tenor)+rr10d(tenor)/2

  return vol

def d25(tenor):
  vol=atm(tenor)+fly25d(tenor)+rr25d(tenor)/2

  return vol

def d75(tenor):
  vol=atm(tenor)+fly25d(tenor)-rr25d(tenor)/2

  return vol

def d90(tenor):
  vol=atm(tenor)+fly10d(tenor)-rr10d(tenor)/2

  return vol

In [280]:
#Interpolation for vol for tenor b/w 0.1, 0.25, 0.5, 0.75 and 0.9 call delta

from scipy import interpolate

def vol(delta,tenor):

  y1=d10(tenor)
  y2=d25(tenor)
  y3=atm(tenor)
  y4=d75(tenor)
  y5=d90(tenor)

  x_points = [0.1, 0.25, 0.5, 0.75, 0.9]
  y_points = [y1,y2,y3,y4,y5]

  tck = interpolate.splrep(x_points, y_points)
  return float(interpolate.splev(delta, tck))

vol(0.1,1)

0.07722499999999997

In [281]:
df

,Unnamed: 0,Arithmetic Average,Unnamed: 2,Geometric Average,Unnamed: 4
0,NaN,Avg Asset,Avg Strike,Avg Asset,Avg Strike
1,Strike,74,NaN,74,NaN
2,Spot,73.32,73.32,73.32,73.32
3,Start Date,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00
4,End Date,2022-06-21 00:00:00,2022-06-22 00:00:00,2022-06-23 00:00:00,2022-06-24 00:00:00
5,Tenor(days),365,365,365,365
6,Tenor(years),1,1,1,1
7,INR Rate(%),0.0425,0.0425,0.0425,0.0425
8,USD Rate(%),0.002,0.002,0.002,0.002
9,Forward,76.3504,76.3504,76.3504,76.3504


In [282]:
#Making Vol Lookup Function

import numpy as np
import math
import scipy
from scipy.stats import norm


def VolArithAsset():
  
  tenor=df._get_value(6,1,True)
  rate=df._get_value(7,1,True)-df._get_value(8,1,True)
  strike=df._get_value(1,1,True)
  spot=df._get_value(2,1,True)
  
  v = atm(tenor)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v



def VolArithStrike():
  
  tenor=df._get_value(6,2,True)
  rate=df._get_value(7,2,True)-df._get_value(8,2,True)
  t=df._get_value(11,2,True)
  spot=df._get_value(2,2,True)
  asset=0
  fwd=spot
  n=round(tenor*365/t)

  v = atm(tenor)

  for j in range(n):
    fwd = fwd * math.exp((rate - (v*v) / 2) * (t/365) + v * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
    asset=asset+fwd

  strike=asset/n


  
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v



def VolGeomAsset():
  
  tenor=df._get_value(6,3,True)
  rate=df._get_value(7,3,True)-df._get_value(8,3,True)
  strike=df._get_value(1,3,True)
  spot=df._get_value(2,3,True)

  


  v = atm(tenor)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v




def VolGeomStrike():
  
  tenor=df._get_value(6,4,True)
  rate=df._get_value(7,4,True)-df._get_value(8,4,True)
  t=df._get_value(11,4,True)
  
  fwd=df._get_value(2,4,True)
  sum=0
  n=round(tenor*365/t)
  v = atm(tenor)

  for j in range(n):
    fwd = fwd * math.exp((rate - (v * v) / 2) * (t/365) + v * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
    x=np.log(fwd)
    sum=sum+x

  avg_x=sum/n
  strike=math.exp(avg_x)

  
  
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v

In [283]:
import numpy as np
import math
#Making Digital Option Pricing Function

#Arithmetic Average of Spot
def AsianArithAvgAsset(strike, spot, tenor, INRr, USDr, vol, t, numsim):
  
  
  rate = INRr-USDr
  
  call_sum = 0.0
  put_sum=0
  if vol==0:
    vol=VolArithAsset()
  else:
    vol=vol
  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    asset=0
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
      asset=asset+fwd

    avg=asset/n

   
    call_payoff=max(avg-strike,0)
    put_payoff=max(strike-avg,0)

    call_sum = call_sum + call_payoff
    put_sum= put_sum + put_payoff

  Call=100*(call_sum / numsim) * math.exp(-rate * tenor)
  Put=100*(put_sum / numsim) * math.exp(-rate * tenor)
  
  return Call,Put


#Arithmetic Average of Strike

def AsianArithAvgStrike(spot, tenor, INRr, USDr, vol, t, numsim):
  
  
  rate = INRr-USDr
  if vol==0:
    vol=VolArithStrike()
  else:
    vol=vol
  call_sum = 0.0
  put_sum=0

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    asset=0
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
      asset=asset+fwd

    avg=asset/n

   
    call_payoff=max(fwd-avg,0)
    put_payoff=max(avg-fwd,0)

    call_sum = call_sum + call_payoff
    put_sum= put_sum + put_payoff

  Call=100*(call_sum / numsim) * math.exp(-rate * tenor)
  Put=100*(put_sum / numsim) * math.exp(-rate * tenor)
  
  return Call,Put



#Geometric AVG of SPot


def AsianGeomAvgAsset(strike, spot, tenor, INRr, USDr, vol, t, numsim):
  
  
  rate = INRr-USDr
  
  if vol==0:
    vol=VolGeomAsset()
  else:
    vol=vol
  call_sum = 0.0
  put_sum=0

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    sum=0
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
      x=np.log(fwd)
      sum=sum+x


    avg_x=sum/n
    avg=math.exp(avg_x)

   
    call_payoff=max(avg-strike,0)
    put_payoff=max(strike-avg,0)

    call_sum = call_sum + call_payoff
    put_sum= put_sum + put_payoff

  Call=100*(call_sum / numsim) * math.exp(-rate * tenor)
  Put=100*(put_sum / numsim) * math.exp(-rate * tenor)
  
  return Call,Put


#Geometric Average of Strike

def AsianGeomAvgStrike(spot, tenor, INRr, USDr, vol, t, numsim):
  
  
  rate = INRr-USDr
  
  if vol==0.0:
    vol=VolGeomStrike()
  else:
    vol=vol
  
  call_sum = 0.0
  put_sum=0.0

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    sum=0
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
      x=np.log(fwd)
      sum=sum+x

    avg_x=sum/n
    avg=math.exp(avg_x)
    
   
    call_payoff=max(fwd-avg,0)
    put_payoff=max(avg-fwd,0)

    call_sum = call_sum + call_payoff
    put_sum= put_sum + put_payoff

  Call=100*(call_sum / numsim) * math.exp(-rate * tenor)
  Put=100*(put_sum / numsim) * math.exp(-rate * tenor)
  
  return Call,Put


In [284]:
df


,Unnamed: 0,Arithmetic Average,Unnamed: 2,Geometric Average,Unnamed: 4
0,NaN,Avg Asset,Avg Strike,Avg Asset,Avg Strike
1,Strike,74,NaN,74,NaN
2,Spot,73.32,73.32,73.32,73.32
3,Start Date,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00
4,End Date,2022-06-21 00:00:00,2022-06-22 00:00:00,2022-06-23 00:00:00,2022-06-24 00:00:00
5,Tenor(days),365,365,365,365
6,Tenor(years),1,1,1,1
7,INR Rate(%),0.0425,0.0425,0.0425,0.0425
8,USD Rate(%),0.002,0.002,0.002,0.002
9,Forward,76.3504,76.3504,76.3504,76.3504


Arithmetic Average for Underlying Price

In [285]:
print('Call and Put Price is',AsianArithAvgAsset(df._get_value(1,1,True),df._get_value(2,1,True),df._get_value(6,1,True),df._get_value(7,1,True),df._get_value(8,1,True),df._get_value(10,1,True),df._get_value(11,1,True),int(df._get_value(12,1,True))))

Call and Put Price is (130.5203447258968, 63.115107677968986)


Arithmetic Average for Strike Price


In [286]:
print('Call and Put Price is',AsianArithAvgStrike(df._get_value(2,2,True),df._get_value(6,2,True),df._get_value(7,2,True),df._get_value(8,2,True),df._get_value(10,2,True),df._get_value(11,2,True),int(df._get_value(12,2,True))))

Call and Put Price is (187.27616115758514, 41.731112861943515)


Geometric Average for Underlying Price


In [287]:
print('Call and Put Price is',AsianGeomAvgAsset(df._get_value(1,3,True),df._get_value(2,3,True),df._get_value(6,3,True),df._get_value(7,3,True),df._get_value(8,3,True),df._get_value(10,3,True),df._get_value(11,3,True),int(df._get_value(12,3,True))))

Call and Put Price is (139.1223542732873, 62.873233799265094)


Geometric Average for Strike Price

In [288]:
print('Call and Put Price is', AsianGeomAvgStrike(df._get_value(2,4,True),df._get_value(6,4,True),df._get_value(7,4,True),df._get_value(8,4,True),df._get_value(10,4,True),df._get_value(11,4,True),int(df._get_value(12,4,True))))

Call and Put Price is (194.0610743897388, 53.05198562860736)
